<a href="https://colab.research.google.com/github/anngook/farmers-note-ml-dev/blob/develop/model/disease_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# 1. 라이브러리 설치 및 데이터 다운로드
# ==============================================================================

!pip install efficientnet_pytorch
!pip install torchmetrics
!pip install kagglehub  # KaggleHub 설치

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import kagglehub # 임포트
import time # ⚠️ 이 부분이 실행되어야 'time.time()'을 사용할 수 있습니다.
import copy # ⚠️ 이 부분이 실행되어야 'copy.deepcopy()'를 사용할 수 있습니다.

from google.colab import drive #

# 다운로드 및 경로 설정
# dataset_download는 다운로드 후 압축을 해제한 데이터셋의 루트 경로를 반환합니다.
# 이 경로는 보통 '/root/.kaggle/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/latest' 와 유사합니다.
path = kagglehub.dataset_download("loki4514/rice-leaf-diseases-detection")

print("다운로드된 데이터셋 경로:", path)

In [2]:
# GPU 사용 설정 (⚠️ 이 부분이 실행되어야 합니다!)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

사용 장치: cuda:0


In [4]:
DATA_ROOT = '/root/.cache/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/versions/8'

print("루트 경로의 내부 목록을 확인합니다:")
print("==================================")
# 'ls -F' 명령어로 DATA_ROOT 내부 목록을 출력합니다.
!ls -F {DATA_ROOT}

루트 경로의 내부 목록을 확인합니다:
Rice_Leaf_AUG/	Rice_Leaf_Diease/


In [5]:
# ==============================================================================
# 2. 데이터셋 경로 설정 (최종 확정 수정)
# ==============================================================================

# path 변수에는 다운로드 루트 경로 '/root/.cache/.../versions/8'이 담겨있다고 가정합니다.
DATA_ROOT = path

# 🚨 이미지에서 확인된 정확한 중첩 폴더 구조를 반영합니다.
# 'Rice_Leaf_Diease' 폴더가 두 번 중복됩니다.
NESTED_ROOT = os.path.join(DATA_ROOT, 'Rice_Leaf_Diease', 'Rice_Leaf_Diease')

# ⚠️ 'train'과 'test' 폴더 이름을 사용합니다. (모두 소문자, 이미지에서 확인 완료)
TRAIN_DIR = os.path.join(NESTED_ROOT, 'train')
VAL_DIR = os.path.join(NESTED_ROOT, 'test')

print(f"최종 TRAIN 경로: {TRAIN_DIR}")
print(f"최종 VAL 경로: {VAL_DIR}")

최종 TRAIN 경로: /root/.cache/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/versions/8/Rice_Leaf_Diease/Rice_Leaf_Diease/train
최종 VAL 경로: /root/.cache/kagglehub/datasets/loki4514/rice-leaf-diseases-detection/versions/8/Rice_Leaf_Diease/Rice_Leaf_Diease/test


In [6]:
# ==============================================================================
# 3. 하이퍼파라미터 및 데이터 변환 설정
# ==============================================================================
from efficientnet_pytorch import EfficientNet

# 하이퍼파라미터
BATCH_SIZE = 32 # 코랩 무료 버전 GPU 메모리에 맞게 조절 (EfficientNet-B0는 32도 가능)
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
MODEL_NAME = 'efficientnet-b0' # B0 선택

# EfficientNet-B0의 권장 입력 이미지 크기
INPUT_SIZE = EfficientNet.get_image_size(MODEL_NAME)

# 데이터 증강 및 정규화
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.RandomResizedCrop(INPUT_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet 통계
    ]),
    'test': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [14]:
# ==============================================================================
# 4. 데이터 로드 및 DataLoader 생성
# ==============================================================================

print("\n데이터 로딩 중...")

image_datasets = {
    'train': datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'test': datasets.ImageFolder(VAL_DIR, data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=BATCH_SIZE, shuffle=True, num_workers=4),
    'test': DataLoader(image_datasets['test'], batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
NUM_CLASSES = len(class_names)

print(f"클래스 개수: {NUM_CLASSES}")
print(f"클래스 이름: {class_names}")
print(f"학습 데이터셋 크기: {dataset_sizes['train']}")
print(f"검증 데이터셋 크기: {dataset_sizes['test']}")


데이터 로딩 중...
클래스 개수: 10
클래스 이름: ['bacterial_leaf_blight', 'brown_spot', 'healthy', 'leaf_blast', 'leaf_scald', 'narrow_brown_spot', 'neck_blast', 'rice_hispa', 'sheath_blight', 'tungro']
학습 데이터셋 크기: 15023
검증 데이터셋 크기: 3422


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
# ==============================================================================
# 5. 모델 설정 (EfficientNet-B0 전이 학습)
# ==============================================================================

model_ft = EfficientNet.from_pretrained(MODEL_NAME, num_classes=NUM_CLASSES).to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


100%|██████████| 20.4M/20.4M [00:00<00:00, 205MB/s]


Loaded pretrained weights for efficientnet-b0


In [9]:
# ==============================================================================
# 6. 모델 학습 함수 (test 키 사용으로 수정)
# ==============================================================================

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}'); print('-' * 10)

        # ⚠️ 'val' 대신 'test' 키를 사용합니다.
        for phase in ['train', 'test']:
            if phase == 'train': model.train()
            else: model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train': scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # ⚠️ 출력 메시지도 'test'로 수정
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # ⚠️ 'test' 단계의 정확도가 최고 기록일 때 모델을 저장합니다.
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'-> New best model found! Saving weights with Acc: {best_acc:.4f}')

    time_elapsed = time.time() - since
    print(f'\n학습 완료! 총 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'최고 검증 정확도: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# ==============================================================================
# 7. 모델 학습 실행 및 저장
# ==============================================================================

# 모델 학습 시작
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=NUM_EPOCHS)


In [11]:
# 현재 할당된 GPU 정보 확인
!nvidia-smi

Wed Oct  8 06:47:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             33W /   70W |    3494MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
# 구글 드라이브 마운트 (저장을 위해 필요합니다)
drive.mount('/content/drive')
MODEL_SAVE_PATH = '/content/drive/MyDrive/efficientnet_rice_disease_best_model.pth'

# 모델 저장
torch.save(model_ft.state_dict(), MODEL_SAVE_PATH)
print(f"\n✅ 모델 가중치가 {MODEL_SAVE_PATH}에 저장되었습니다.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

✅ 모델 가중치가 /content/drive/MyDrive/efficientnet_rice_disease_best_model.pth에 저장되었습니다.
